In [106]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from fuzzywuzzy import fuzz
from datetime import date
import datetime as dt
import pandas as pd
import numpy as np
import random
from selenium.webdriver.chrome.options import Options
import re
from tqdm import tqdm
import warnings
import selenium
warnings.filterwarnings("ignore")

In [107]:
driver = webdriver.Chrome()
driver.get("https://www.nwslsoccer.com/schedule")

In [108]:
row_text = [val.text for val in driver.find_elements("class name","row")]

In [109]:
links = pd.Series([val.get_attribute("href") for val in driver.find_elements("xpath","//a[@href]")])
game_urls = links[links.str.contains("game")&links.str.contains("vs")&~links.str.contains("stats")].drop_duplicates()
game_info = game_urls.apply(lambda x: x.split("/")[-1])

In [110]:
game_date_series = pd.to_datetime(game_info.apply(lambda x: x[-10:]))
home_team_series = game_info.apply(lambda x: x.split("-vs-")[0])
away_team_series = game_info.apply(lambda x: x.split("-vs-")[1][:-11])

home_team_series = home_team_series.apply(lambda x: re.sub("-"," ",x))
away_team_series = away_team_series.apply(lambda x: re.sub("-"," ",x))


In [111]:
len(game_urls)

182

In [113]:
row_text_games = pd.Series(row_text)[pd.Series(row_text).str.contains("EVENT")&~pd.Series(row_text).str.contains("2024")].drop_duplicates()

In [114]:
row_text_df = row_text_games.apply(lambda x: pd.Series(x.split("\n")))

In [115]:
stadiums = ['CPKC Stadium', 'Lynn Family Stadium', 'WakeMed Soccer Park',
       'America First Field', 'Lumen Field', 'BMO Stadium',
       'Inter&Co Stadium', 'SeatGeek Stadium', 'Audi Field',
       'Shell Energy Stadium', 'Snapdragon Stadium', 'Providence Park',
       'PayPal Park','Shell Energy Stadium',
       'America First Field', 'WakeMed Soccer Park',
       'Lynn Family Stadium', 'SeatGeek Stadium', 'Red Bull Arena',
       'PayPal Park', 'Inter&Co Stadium', 'Audi Field', 'CPKC Stadium',
       'Providence Park', 'Lumen Field', 'BMO Stadium',
       'Snapdragon Stadium']

In [116]:
location = pd.concat([row_text_df[row_text_df[7].isin(stadiums)][7],row_text_df[row_text_df[5].isin(stadiums)][5]])

In [123]:
gametime = list(row_text_df[4].values)

In [117]:
full_df = pd.DataFrame(zip(game_date_series,home_team_series,away_team_series),columns=["GAME_DATE","HOME_TEAM","AWAY_TEAM"])

In [126]:
full_df["LOCATION"] = list(location)
full_df["GAME_TIME"] = list(gametime)
full_df.loc[~full_df["GAME_TIME"].str.contains("PM"),"GAME_TIME"] = "-"
full_df

,GAME_DATE,HOME_TEAM,AWAY_TEAM,LOCATION,GAME_TIME
0,2024-03-16,kansas city current,portland thorns fc,CPKC Stadium,-
1,2024-03-16,racing louisville fc,orlando pride,Lynn Family Stadium,-
2,2024-03-16,north carolina courage,houston dash,WakeMed Soccer Park,-
3,2024-03-16,utah royals fc,chicago red stars,America First Field,-
4,2024-03-17,seattle reign,washington spirit,Lumen Field,-
...,...,...,...,...,...
177,2024-11-02,orlando pride,seattle reign,Inter&Co Stadium,5:00 PM ET
178,2024-11-02,north carolina courage,washington spirit,WakeMed Soccer Park,7:30 PM ET
179,2024-11-02,houston dash,bay fc,Shell Energy Stadium,9:30 PM ET
180,2024-11-03,chicago red stars,kansas city current,SeatGeek Stadium,3:00 PM ET


In [127]:
full_df.to_csv("nwsl_soccer_schedule.csv",index=False)